In [2]:
#80/20 train/test split of model 

# @article{Hug2020,
#   doi = {10.21105/joss.02174},
#   url = {https://doi.org/10.21105/joss.02174},
#   year = {2020},
#   publisher = {The Open Journal},
#   volume = {5},
#   number = {52},
#   pages = {2174},
#   author = {Nicolas Hug},
#   title = {Surprise: A Python library for recommender systems},
#   journal = {Journal of Open Source Software}
# }



import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

df = pd.read_csv('joined_dataset.csv', low_memory=False)

reader = Reader(rating_scale=(1, 10)) 
data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)

model = SVD()

In [3]:
from surprise.model_selection import cross_validate

cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6322  1.6337  1.6426  1.6367  1.6318  1.6354  0.0040  
MAE (testset)     1.2591  1.2596  1.2686  1.2599  1.2624  1.2619  0.0035  
Fit time          6.62    6.92    6.84    6.84    7.03    6.85    0.13    
Test time         0.53    0.89    0.94    0.57    0.89    0.76    0.18    


{'test_rmse': array([1.63220043, 1.63369043, 1.64261287, 1.63667123, 1.63178872]),
 'test_mae': array([1.2591123 , 1.2595566 , 1.26855778, 1.25991446, 1.26244495]),
 'fit_time': (6.619834661483765,
  6.922223806381226,
  6.84015679359436,
  6.844214200973511,
  7.027630805969238),
 'test_time': (0.5338289737701416,
  0.8902616500854492,
  0.9363884925842285,
  0.5674099922180176,
  0.8942067623138428)}

In [4]:
# Random predictor baseline
from surprise import NormalPredictor

random_model = NormalPredictor()  # Random predictions based on normal distribution
cross_validate(random_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.5018  2.5083  2.4945  2.4964  2.4952  2.4992  0.0052  
MAE (testset)     1.9977  2.0013  1.9915  1.9946  1.9876  1.9945  0.0047  
Fit time          0.44    0.62    0.59    0.60    0.59    0.57    0.07    
Test time         0.33    0.92    0.34    0.36    0.37    0.46    0.23    


{'test_rmse': array([2.50178154, 2.50825715, 2.49445128, 2.49636401, 2.49524615]),
 'test_mae': array([1.99768136, 2.00131346, 1.99146921, 1.99461731, 1.98764884]),
 'fit_time': (0.43929076194763184,
  0.6213939189910889,
  0.5862705707550049,
  0.6023659706115723,
  0.5927672386169434),
 'test_time': (0.32723188400268555,
  0.922950267791748,
  0.34299492835998535,
  0.3572525978088379,
  0.3670175075531006)}

In [5]:
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import cross_validate

# Custom baseline algorithm class
class MeanPredictor(AlgoBase):
    def __init__(self):
        # Call the base constructor
        AlgoBase.__init__(self)

    def fit(self, trainset):
        # This method is used to fit the algorithm to the training set
        # Store the global mean rating for predictions
        AlgoBase.fit(self, trainset)
        self.global_mean = trainset.global_mean
        return self

    def estimate(self, u, i):
        # This method estimates the rating for user u and item i
        # Here we simply return the global mean
        return self.global_mean


# Create the baseline model (MeanPredictor)
baseline_model = MeanPredictor()

# Perform cross-validation
cv_results = cross_validate(baseline_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Output the cross-validation results
print(cv_results)


Evaluating RMSE, MAE of algorithm MeanPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8412  1.8383  1.8453  1.8376  1.8442  1.8413  0.0031  
MAE (testset)     1.4931  1.4918  1.4986  1.4910  1.4967  1.4942  0.0029  
Fit time          0.21    0.35    0.35    0.35    0.35    0.32    0.06    
Test time         0.25    0.23    0.93    0.93    0.91    0.65    0.33    
{'test_rmse': array([1.84124197, 1.83828821, 1.84527161, 1.83759433, 1.84423261]), 'test_mae': array([1.49309297, 1.4917903 , 1.49855347, 1.49104539, 1.49668362]), 'fit_time': (0.20583438873291016, 0.3493316173553467, 0.3504791259765625, 0.35009145736694336, 0.35471200942993164), 'test_time': (0.2501943111419678, 0.23462271690368652, 0.9346303939819336, 0.9334135055541992, 0.9073429107666016)}


In [6]:
from surprise import KNNBaseline
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

# Load your dataset
df = pd.read_csv('joined_dataset.csv', low_memory=False)

# Filter users with fewer than a threshold number of ratings
min_ratings = 3
df_filtered = df.groupby('User-ID').filter(lambda x: len(x) >= min_ratings)

# Filter items (books) with fewer than a threshold number of ratings
min_item_ratings = 10
df_filtered = df_filtered.groupby('ISBN').filter(lambda x: len(x) >= min_item_ratings)


reader = Reader(rating_scale=(df_filtered['Book-Rating'].min(), df['Book-Rating'].max()))
data = Dataset.load_from_df(df_filtered[['User-ID', 'ISBN', 'Book-Rating']], reader)

# Set the similarity options
sim_options = {
    'name': 'pearson_baseline',  # Use Pearson correlation
    'user_based': True  # If True, computes similarities between users. False for items.
}

# Initialize the KNNBaseline algorithm
knn_baseline = KNNBaseline(sim_options=sim_options)

# Perform cross-validation
cv_results_knn = cross_validate(knn_baseline, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Output results
print(cv_results_knn)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.7418  1.7155  1.7397  1.7531  1.7316  1.7363  0.0125  
MAE (testset)     1.3249  1.3095  1.3245  1.3292  1.3244  1.3225  0.0067  
Fit time          49.82   27.95   28.48   18.91   20.77   29.19   10.99   
Test time         1.30    1.45    1.31    1.27    3.4

In [10]:
# Made svd_model, prevent overwriting
import pickle

with open('svd_model.pkl', 'wb') as file:
    pickle.dump(model, file)